In [18]:
'''
Pull price data from Marketstack API
Created JR 09/10/2024
Requires API key from Marketstack. 
'''

# required packages
import requests
import pandas as pd
import os
from dotenv import load_dotenv

# get API key from .env file
load_dotenv()
API_KEY = os.getenv('Marketstack')

# Define the securities and date range
symbols = 'HSBA.XLON'  
end_date = "2024-09-30"  
start_date = "2024-06-30"  

# API URL from marketstack
url = 'http://api.marketstack.com/v1/eod'

# Get results and return dataframe. Created using examples from https://marketstack.com/documentation
def get_eod_data(symbols, start_date, end_date, api_key):
    
    all_data = []  # List to store all records
    limit = 1000 # confirmed max 1000 via docs
    offset = 0

    while True:
        # Define parameters for the API request
        querystring = {
            'access_key': api_key,
            'symbols': symbols,
            'date_from': start_date,
            'date_to': end_date,
            'limit': limit,
            'offset': offset  
        }

        # Make the request
        response = requests.get(url, params=querystring)

        # Check all data has been received
        if response.status_code == 200:
            data = response.json().get('data', [])
            if not data:
                break  

            # Append new data to the list
            all_data.extend(data)

            # Increment the offset to get the next set of data
            offset += limit

        else:
            print(f"Error: {response.status_code}, {response.text}")
            break

    # Convert the collected data to a pandas DataFrame
    return pd.DataFrame(all_data)

# Fetch all paginated data
df = get_eod_data(symbols, start_date, end_date, API_KEY)

# Show the first few rows of the combined data
print(df.head)

<bound method NDFrame.head of Empty DataFrame
Columns: []
Index: []>


In [17]:
print(df)


df.to_csv('combined_eod_data1.csv', index=False)

    open   high    low  close      volume adj_high adj_low  adj_close  \
0  653.8  657.6  648.6  650.3  12350267.0     None    None      650.3   
1  643.1  655.5  643.1  653.0  33500611.0     None    None      653.0   
2  652.6  653.8  648.1  651.3  20657846.0     None    None      651.3   
3  651.7  663.2  647.1  652.6  11126299.0     None    None      652.6   
4  660.4  664.5  654.6  657.9  30000246.0     None    None      657.9   
5  655.4  661.5  651.1  659.0  15631541.0     None    None      659.0   
6  650.0  660.6  648.2  657.8  18211081.0     None    None      657.8   

  adj_open adj_volume  split_factor  dividend     symbol exchange  \
0     None       None           1.0       0.0  HSBA.XLON     XLON   
1     None       None           1.0       0.0  HSBA.XLON     XLON   
2     None       None           1.0       0.0  HSBA.XLON     XLON   
3     None       None           1.0       0.0  HSBA.XLON     XLON   
4     None       None           1.0       0.0  HSBA.XLON     XLON   
5

In [9]:

# required packages
import requests
import pandas as pd
import os
from dotenv import load_dotenv

# get API key from .env file
load_dotenv()
API_KEY = os.getenv('Marketstack')

# Define the securities and date range
symbols = ['HSBA.XLON', 'RDSB.XAMS']  # Now a list of symbols
end_date = "2024-09-30"  
start_date = "2019-06-30"  

# API URL from marketstack
url = 'http://api.marketstack.com/v1/eod'

# Function to fetch EOD data for a single symbol
def get_eod_data(symbol, start_date, end_date, api_key):
    
    all_data = []  # List to store all records
    limit = 1000  # confirmed max 1000 via docs
    offset = 0

    while True:
        # Define parameters for the API request
        querystring = {
            'access_key': api_key,
            'symbols': symbol,
            'date_from': start_date,
            'date_to': end_date,
            'limit': limit,
            'offset': offset  
        }

        # Make the request
        response = requests.get(url, params=querystring)

        # Check all data has been received
        if response.status_code == 200:
            data = response.json().get('data', [])
            if not data:
                break  # No more data, exit the loop

            # Append new data to the list
            all_data.extend(data)

            # Increment the offset to get the next set of data
            offset += limit

        else:
            print(f"Error: {response.status_code}, {response.text}")
            break

    # Convert the collected data to a pandas DataFrame
    df = pd.DataFrame(all_data)
    df['symbol'] = symbol  # Add a column for the symbol

    return df

# Initialize an empty DataFrame to store all the data
all_symbols_df = pd.DataFrame()

# Loop through each symbol and fetch its data
for symbol in symbols:
    symbol_df = get_eod_data(symbol, start_date, end_date, API_KEY)
    all_symbols_df = pd.concat([all_symbols_df, symbol_df], ignore_index=True)

# Display the first few rows of the combined data
print(all_symbols_df.head())

# Optional: save the combined data to a CSV file
# all_symbols_df.to_csv('combined_eod_data.csv', index=False)


    open   high    low  close      volume adj_high adj_low  adj_close  \
0  653.8  657.6  648.6  650.3  12350267.0     None    None      650.3   
1  643.1  655.5  643.1  653.0  33500611.0     None    None      653.0   
2  652.6  653.8  648.1  651.3  20657846.0     None    None      651.3   
3  651.7  663.2  647.1  652.6  11126299.0     None    None      652.6   
4  660.4  664.5  654.6  657.9  30000246.0     None    None      657.9   

  adj_open adj_volume  split_factor  dividend     symbol exchange  \
0     None       None           1.0       0.0  HSBA.XLON     XLON   
1     None       None           1.0       0.0  HSBA.XLON     XLON   
2     None       None           1.0       0.0  HSBA.XLON     XLON   
3     None       None           1.0       0.0  HSBA.XLON     XLON   
4     None       None           1.0       0.0  HSBA.XLON     XLON   

                       date  
0  2023-10-18T00:00:00+0000  
1  2023-10-17T00:00:00+0000  
2  2023-10-16T00:00:00+0000  
3  2023-10-13T00:00:00+000

In [10]:
print(all_symbols_df)

all_symbols_df.to_csv('combined_eod_data_v2.csv', index=False)

    open   high    low  close      volume adj_high adj_low  adj_close  \
0  653.8  657.6  648.6  650.3  12350267.0     None    None      650.3   
1  643.1  655.5  643.1  653.0  33500611.0     None    None      653.0   
2  652.6  653.8  648.1  651.3  20657846.0     None    None      651.3   
3  651.7  663.2  647.1  652.6  11126299.0     None    None      652.6   
4  660.4  664.5  654.6  657.9  30000246.0     None    None      657.9   
5  655.4  661.5  651.1  659.0  15631541.0     None    None      659.0   
6  650.0  660.6  648.2  657.8  18211081.0     None    None      657.8   

  adj_open adj_volume  split_factor  dividend     symbol exchange  \
0     None       None           1.0       0.0  HSBA.XLON     XLON   
1     None       None           1.0       0.0  HSBA.XLON     XLON   
2     None       None           1.0       0.0  HSBA.XLON     XLON   
3     None       None           1.0       0.0  HSBA.XLON     XLON   
4     None       None           1.0       0.0  HSBA.XLON     XLON   
5